# Mapping seismic stations in the Himalayas with Numpy and Matplotlib
## Or reading datasets with custom dtypes and plotting Earth-based data with basemap

In this exercise, we consider loading measurement files with the format:

<pre>
station  lat    long   elev 
BIRA	26.4840	87.2670	0.0120
BUNG	27.8771	85.8909	1.1910
etc...
</pre>

These are seismic measurement stations in the Himalaya, with the elevation indicated in km.  Data with a structure such as this is common in many disciplines, and because we have a combination of text and numerical fields, we can't directly load it into a regular numpy array.

But we can use pandas' ability to [read tabular data files](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) to load it into a DataFrame.

If you have the basemap matplotlib toolkit installed, at the end of this example we will show a real Earth map and overlay the station locations on top of that.

We start by configuring matplotlib support and loading the required modules.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Now, we load the data using pandas

In [ ]:
import os
data_fname = os.path.join('data', 'stations.txt')
tab = pd.read_csv(data_fname, delim_whitespace=True)
tab

You can now print out key information about the stations. In addition to the information below, print the mean latitude of the stations (which you should be able to do with one command).

In [ ]:
ptitle = 'Seismic stations in the Himalaya'
print(ptitle)
print('Stations:', tab.station)
print('Elevations (km):', tab.elev)
print('First station:', tab[:1])

Now we can use `matplotlib` to plot the relative locations of the stations. Create such a plot using `plt.scatter` (the $x$ "coordinate" should be the longitude, and the $y$ "coordinate" the latitude). Improve the appearance of the plot by

1. Scaling the size of the points according to the elevation
2. Coloring the points according to the elevation.
3. Adding the "name" of the station as text to the plot, next to the point location.

In [ ]:
f1, ax = plt.subplots(figsize = (8,5))

# Make the size of the circles proportional to the elevation

# Add scatter plot
s = ax.scatter(...)

# The colorbar must be associated with the return value of scatter()
f1.colorbar(s)
ax.set_title(ptitle)

# Now add text labels for all the stations.  

# Note: when accessing single elements of the data frame, the named field
# syntax doesn't work and we must access the fields via record.station
for i, record in tab.iterrows():
    # Code here
    pass

To improve the plot further, we can use the `matplotlib` `basemap` toolkit. If you don't have it already installed and are using conda, then try

```bash
conda install basemap
```

Work through the code below to understand how the points are mapped onto the image and contours.

In [ ]:
try:
    from mpl_toolkits.basemap import Basemap
except ImportError:
    print("Basemap Not Available, no further plotting.")
else:
    # Draw the stations on a real map of the Earth.
    # Find boundaries 
    lon0 = 0.995*tab.long.min()
    lon1 = 1.01*tab.long.max()
    lat0 = 0.995*tab.lat.min()
    lat1 = 1.01*tab.lat.max()
    # Geographic grid to draw
    parallels = np.linspace(lat0, lat1, 5)
    meridians = np.linspace(lon0, lon1, 5)

    # Resolution of the basemap to load ('f' is *very* expensive)
    resolution = 'i' # intermediate resolution for map info

    f2, ax2 = plt.subplots(figsize=(10,6))
    m = Basemap(lon0, lat0, lon1, lat1, resolution=resolution, ax=ax2)
    m.drawcountries(color=(1,1,0))  # country boundaries yellow
    m.drawrivers(color=(0,1,1))  # rivers in cyan
    m.bluemarble()  # NASA bluemarble image
    m.drawparallels(parallels, labels=[1,0,0,0], fmt='%.2f')
    m.drawmeridians(meridians, labels=[0,0,0,1], fmt='%.2f')
    s = m.scatter(tab.long, tab.lat, s=sizes, c=tab.elev, zorder=10, alpha=0.6)
    f2.colorbar(s)
    ax2.set_title(ptitle)
    for i, record in tab.iterrows():
        ax2.text( record.long + 0.05, record.lat + 0.05, record.station, 
                weight='bold', color='yellow', zorder=10)